# WEEK 3 ASSIGNMENT

## Segmenting and Clustering Neighborhoods in Toronto

#### First import all the libraries you'll need

In [1]:
import numpy as np
import pandas as pd 
import codecs
from bs4 import BeautifulSoup

#### Get the wikipedia page you want to extract data from

In [4]:
!wget https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

--2020-05-05 01:30:21--  https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M
Resolving en.wikipedia.org (en.wikipedia.org)... 208.80.153.224, 2620:0:860:ed1a::1
Connecting to en.wikipedia.org (en.wikipedia.org)|208.80.153.224|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 52231 (51K) [text/html]
Saving to: ‘List_of_postal_codes_of_Canada:_M.1’

100%[======================================>] 52,231      --.-K/s   in 0.002s  

2020-05-05 01:30:21 (21.6 MB/s) - ‘List_of_postal_codes_of_Canada:_M.1’ saved [52231/52231]



#### Use beautiful soup to extract the table with the data from the page and convert to dataframe

In [6]:
f = codecs.open('List_of_postal_codes_of_Canada:_M', encoding='utf-8')
html = f.read()
soup = BeautifulSoup(html, 'html.parser')
df = (pd.read_html(str(soup.table))[0])

df.head()

,Postal code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


#### Now the 'Not assigned' Borough will be removed

In [7]:
Toronto_df = Toronto_df[Toronto_df['Borough'] != "Not assigned"].reset_index(drop=True) 
Toronto_df.head()

,Postal code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park / Harbourfront
3,M6A,North York,Lawrence Manor / Lawrence Heights
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


Observe the dataframe above, wikipedia has actually rearranged the dataframe and has joined the Neighborhoods accordingly. But I will still run the necessary codes to avoid loosing marks.

#### Next, drop duplicates after merging the neighbourhoods with the same Postcode.

In [21]:
aggregations = {
    'Neighborhood':lambda x:', '.join(x)}
df_grp = df.groupby('Postal code').agg(aggregations).reset_index()
df_drop = df.drop('Neighborhood',axis=1)
df_merge= pd.merge(df_drop, df_grp, on='Postal code', how='left')
df_duplicate = df_merge.drop_duplicates()
df_duplicate

,Postal code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park / Harbourfront
3,M6A,North York,Lawrence Manor / Lawrence Heights
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,Malvern / Rouge
7,M3B,North York,Don Mills
8,M4B,East York,Parkview Hill / Woodbine Gardens
9,M5B,Downtown Toronto,Garden District / Ryerson


In [18]:
df.shape

(103, 3)